In [3]:
import sys

sys.path.append('..')
from src.tools.print_sysinfo import print_env
print_env()

========== System Information ==========
DATE : 2023-08-31
Pyton Version : 3.9.17
PyTorch Version : 2.0.1
OS : Windows 10
CPU spec : AMD64 Family 25 Model 33 Stepping 2, AuthenticAMD
RAM spec : 31.88 GB
Device 0:
Name: NVIDIA GeForce RTX 3080 Ti
Total Memory: 12288.0 MB
Driver Version: 536.23


In [4]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from src.tools.rle_encoder import rle_encode
from src.data.dataset import SourceDataset, TargetDataset
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
import torch
import albumentations as A
from albumentations.core.transforms_interface import ImageOnlyTransform

def fisheye_circular_transform_torch(image, mask=None, fov_degree=200, focal_scale=4.5):
    _, h, w = image.shape
    
    # Convert degrees to radians using torch tensor
    radian_conversion = torch.tensor(np.pi/180, dtype=image.dtype, device=image.device)
    
    
    # Calculate the focal length using the given FOV
    f = w / (2 * torch.tan(0.5 * fov_degree * radian_conversion))
    f_scaled = f * focal_scale
    
    # Meshgrid for coordinates
    x = torch.linspace(-w//2, w//2, w).repeat(h, 1)
    y = torch.linspace(-h//2, h//2, h).unsqueeze(1).repeat(1, w)
    r = torch.sqrt(x*x + y*y)
    theta = torch.atan2(y, x)
    
    # Apply fisheye transformation
    r_fisheye = f_scaled * torch.atan(r / f_scaled)
    x_fisheye = (w // 2 + r_fisheye * torch.cos(theta)).long()
    y_fisheye = (h // 2 + r_fisheye * torch.sin(theta)).long()
    
    # Create masks for valid coordinates
    valid_coords = (x_fisheye >= 0) & (x_fisheye < w) & (y_fisheye >= 0) & (y_fisheye < h)
    
    # Initialize output images
    new_image = torch.zeros_like(image)
    if mask is not None:
        new_mask = torch.zeros_like(mask)
    else:
        new_mask = None
    
    # Assign values
    new_image[:, valid_coords] = image[:, y_fisheye[valid_coords], x_fisheye[valid_coords]]
    if mask is not None:
        new_mask[:, valid_coords] = mask[:, y_fisheye[valid_coords], x_fisheye[valid_coords]]
    
    return new_image, new_mask

class FisheyeTransform(ImageOnlyTransform):
    def __init__(self, fov_degree=200, focal_scale=4.5, always_apply=False, p=1.0):
        super(FisheyeTransform, self).__init__(always_apply, p)
        self.fov_degree = fov_degree
        self.focal_scale = focal_scale

    def apply(self, image, **params):
        image_tensor = torch.tensor(image).permute(2, 0, 1).float()
        transformed_image, _ = fisheye_circular_transform_torch(image_tensor, fov_degree=self.fov_degree, focal_scale=self.focal_scale)
        return transformed_image.permute(1, 2, 0).byte().numpy()

    def apply_to_mask(self, mask, **params):
        mask_tensor = torch.tensor(mask).unsqueeze(0).float()
        _, transformed_mask = fisheye_circular_transform_torch(mask_tensor, fov_degree=self.fov_degree, focal_scale=self.focal_scale)
        return transformed_mask.squeeze(0).byte().numpy()

In [16]:
augmentation = A.Compose(
    [
        FisheyeTransform(p=0.2),
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]
)


transform = A.Compose(
    [   
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]
)

In [17]:


train_dataset = SourceDataset(csv_file='train_source.csv', transform=augmentation, is_training=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

valid_dataset = SourceDataset(csv_file='val_source.csv', transform=transform, is_training=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [18]:
import math

def compute_iou(pred, target, num_classes):
    iou_list = []
    pred = pred.view(-1)
    target = target.view(-1)

    # For classes excluding the background
    for cls in range(num_classes - 1):  # We subtract 1 to exclude the background class
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds[target_inds]).sum().float()
        union = (pred_inds + target_inds).sum().float()
        if union == 0:
            iou_list.append(float('nan'))  # If there is no ground truth, do not include in evaluation
        else:
            iou_list.append((intersection / union).item())
    return iou_list

def compute_mIoU(preds, labels, num_classes=13):
    iou_list = compute_iou(preds, labels, num_classes)
    valid_iou_list = [iou for iou in iou_list if not math.isnan(iou)]
    mIoU = sum(valid_iou_list) / len(valid_iou_list)
    return mIoU


In [19]:
# FCN with ResNet50 backbone:

from torchvision.models.segmentation import fcn_resnet50
model = fcn_resnet50(pretrained=True)
model.classifier[4] = nn.Conv2d(512, 13, kernel_size=(1, 1), stride=(1, 1))
model.aux_classifier[4] = nn.Conv2d(256, 13, kernel_size=(1, 1), stride=(1, 1))

# 사전 학습된 모델의 모든 레이어를 고정
for param in model.parameters():
    param.requires_grad = False
    
for param in model.classifier[4].parameters():
    param.requires_grad = True
    
for param in model.aux_classifier[4].parameters():
    param.requires_grad = True
    
model.to(device)

c:\Users\master\miniconda3\envs\vision_task\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\master\miniconda3\envs\vision_task\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [20]:
from torch.optim.lr_scheduler import StepLR


# 1. 모델 불러오기
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)
elif torch.cuda.device_count() == 1:
    print(f"Using only 1 GPU!")
    model.to(device)
else:
    print(f"Using CPU")
    model.to(device)

# 2. 데이터 준비 (여기서는 간략하게 표현합니다)
#train_loader, val_loader = prepare_target_domain_dataloaders()

# 3. 학습 설정
criterion = nn.CrossEntropyLoss() # 예시로 CrossEntropyLoss를 사용합니다
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001) # 작은 learning rate 사용

# Learning rate scheduler 설정
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)


# 4. 학습
num_epochs = 1000
train_losses = []
train_mIoUs = []
val_mIoUs = []

# Early stopping 관련 설정
patience = 20  # 10번의 epoch 동안 성능 향상이 없을 경우 학습 중단
no_improve_epochs = 0  # 성능 향상이 없는 epoch의 횟수
best_mIoU = 0.0  # 최고의 검증 mIoU 저장

# 보조 출력에 대한 손실 가중치
aux_loss_weight = 0.4

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_iou = 0.0
    num_batches = 0
    num_images = 0
    
    for images, masks in tqdm(train_loader):
        
        images = images.float().to(device)
        masks = masks.long().to(device)
        num_images += images.size(0)
        
        optimizer.zero_grad()
        outputs = model(images)
        
        # 주 출력에 대한 손실 계산
        main_loss = criterion(outputs['out'], masks)
        
        # 보조 출력에 대한 손실 계산
        aux_loss = criterion(outputs['aux'], masks)

        # 두 손실을 결합
        loss = main_loss + aux_loss_weight * aux_loss
        loss.backward()
        optimizer.step()
        
        total_loss += num_images * loss.item()
        _, predicted = outputs['out'].max(1)
        total_iou += compute_mIoU(predicted, masks)
        num_batches += 1
    
    avg_loss = total_loss / num_images
    avg_train_mIoU = total_iou / num_images
    train_losses.append(avg_loss)
    train_mIoUs.append(avg_train_mIoU)
    print(f"Epoch {epoch + 1} - Training Loss: {avg_loss:.4f}, Training mIoU: {avg_train_mIoU:.4f}")

    
    # 5. 검증 (간략하게 표현)
    with torch.no_grad():
        model.eval()
        total_iou = 0
        num_images = 0
        for images, masks in tqdm(valid_loader):
            images = images.float().to(device)
            masks = masks.long().to(device)
        
            outputs = model(images)
            _, predicted = outputs['out'].max(1)
            total_iou += compute_mIoU(predicted, masks)
            num_images += images.size(0)
        avg_mIoU = total_iou / num_images
        print(f"Epoch {epoch + 1}, mIoU: {avg_mIoU:.4f}")


    # 학습률 업데이트
    scheduler.step()

    
    # Early stopping 검사
    if avg_mIoU > best_mIoU:
        best_mIoU = avg_mIoU
        # 최적의 모델 저장
        torch.save(model.state_dict(), 'best_model.pth')
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print("Early stopping triggered!")
            # 최적의 모델 불러오기
            model.load_state_dict(torch.load('best_model.pth'))
            break

Using only 1 GPU!


100%|██████████| 69/69 [02:12<00:00,  1.92s/it]


Epoch 1 - Training Loss: 69.0000, Training mIoU: 0.0045


100%|██████████| 30/30 [00:19<00:00,  1.53it/s]


Epoch 1, mIoU: 0.0133


...

Epoch 52, mIoU: 0.0175


100%|██████████| 69/69 [02:22<00:00,  2.06s/it]


Epoch 53 - Training Loss: 37.5749, Training mIoU: 0.0084


100%|██████████| 30/30 [00:20<00:00,  1.47it/s]

Epoch 53, mIoU: 0.0175
Early stopping triggered!


In [21]:
test_dataset = TargetDataset(csv_file='./test.csv', transform=transform, is_training=False)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [24]:
with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        outputs = model(images)['out']
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()
        # batch에 존재하는 각 이미지에 대해서 반복
        for pred in outputs:
            pred = pred.astype(np.int32)
            pred = Image.fromarray(pred) # 이미지로 변환
            pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
            pred = np.array(pred) # 다시 수치로 변환
            # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
            for class_id in range(12):
                class_mask = (pred == class_id).astype(np.int32)
                if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
                    mask_rle = rle_encode(class_mask)
                    result.append(mask_rle)
                else: # 마스크가 존재하지 않는 경우 -1
                    result.append(-1)

100%|██████████| 119/119 [01:54<00:00,  1.04it/s]


In [25]:
submit = pd.read_csv('../data/raw/sample_submission.csv')
submit['mask_rle'] = result
submit

,id,mask_rle
0,TEST_0000_class_0,77015 22 77975 22 78927 47 79887 47 80834 56 8...
1,TEST_0000_class_1,-1
2,TEST_0000_class_2,284 64 562 9 1244 64 1522 9 2204 64 2482 9 316...
3,TEST_0000_class_3,-1
4,TEST_0000_class_4,-1
...,...,...
22771,TEST_1897_class_7,153511 9 154471 9 155431 9 156387 13 157347 13...
22772,TEST_1897_class_8,57 197 1017 197 1977 197 2937 197 3897 197 485...
22773,TEST_1897_class_9,-1
22774,TEST_1897_class_10,-1


In [26]:
submit.to_csv('./fcn_resnet50_feature_extraction.csv', index=False)